# 기본 세팅

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import numpy as np
import random
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)




import warnings

import random

# 모든 경고 메시지 끄기
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'NanumBarunGothic' # 나눔바른고딕 적용하기


In [26]:
actual_result = pd.read_csv('/content/drive/MyDrive/actual_result.csv')

def count_custom_anomalies(data, anomality, prediction, min_temp, max_temp):
    # 조건에 해당하는 행의 개수를 세는 함수
    def count_custom_rows(group):
        return len(group[(group[anomality] == -1) &
                         (group['날짜'] > group[prediction]) &
                         (group['최저기온(°C)'] > min_temp) &
                         (group['최저기온(°C)'] < max_temp)])

    # '연', '지점명'으로 그룹화하고 각 그룹에 대해 함수 적용
    custom_counts = data.groupby(['연', '지점명']).apply(count_custom_rows).reset_index(name='custom_count')

    return custom_counts


In [5]:
def region(x,y,fruit):
  result = {}
  condition = (actual_result['지점명']==x)
  condition2 = (actual_result['isoforest_anomality']==-1)

  # 조건1: '지점명'이 region이고 조건2: 'isoforest_anomality'가 -1인 데이터 추출
  filtered_data = actual_result[(condition) & (condition2)]

  # '연' 별로 그룹화하고 '최저기온(°C)' 열에 대해 10 백분위수와 90 백분위수를 계산
  percentiles = filtered_data['최저기온(°C)'].quantile([y[0], y[1]]).tolist() ; percentiles

  custom_counts =count_custom_anomalies(actual_result[condition], 'isoforest_anomality', fruit , percentiles[0], percentiles[1])
  result['lambda'] = custom_counts.groupby('지점명')['custom_count'].mean().tolist()[0]
  result['updown']= percentiles
  return(result)

In [44]:
# Grid 만들기

upper = [0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55 , 0.6, 0.65 , 0.7, 0.75, 0.8, 0.85, 0.95]

grid = []

for i in upper :
  percentile = [0.1,i]
  grid.append(percentile)

# Insurance Risk Model Function

In [41]:
def simulation(n_join, join, out, claim , T , a0 , unit , c  , up, down):

  # n_join : 보험 가입자의 수 : 100
  # join : 보험 가입의 lambda : 10
  # out : 보험 해지의 lambda : 5
  # claim : (한도 <= 기온 <= 지수) 보험금 지급 claim의 lambda
  # T : 기간 설정
  # a0 : 보험사의 초기 자본금 = 100000000
  # unit : 지수 당 보험 지급액 단위 = 100
  # c : 납부하는 보험금 = 20

  # 1 . 기본 세팅

  t = 0       # 시간은 0으로 설정
  n = n_join  # 보험 가입자의 수
  a = a0      # 현재 보험 회사의 자본
  give = []
  rate = join + n * out + n * claim
  tE = np.random.exponential(scale=1/rate)

  # 시점 T 까지의 보험 시스템

  while tE <= T :

    a = a + n*c*(tE-t)  # 시간 당 발생하는 보험료
    t = tE
    p1 = join / (join + n * out + n * claim)
    p2 = (n * out) / (join + n * out + n * claim)
    p3 =  (n * claim)/ (join + n * out + n * claim)

    J = random.choices([1, 2, 3], weights=[p1, p2, p3], k=1)[0] # import random

    if J == 1:
      n = n + 1

    elif J == 2:
      n = n - 1

    elif J == 3:
      temperature = random.uniform(down, up) # 지수형 보험금 계산을 위한 온도 샘플링
      money = (up-temperature)* unit
      give. append(money)
      a = a - money

    rate = join + n * out + n * claim
    tE = t + np.random.exponential(scale=1/rate)

  result = {}
  result['보험사 자본금'] = a
  #result['보험지금액'] = give
  result['평균지급액'] = sum(give)/(len(give)+ 0.000001)
  result['지급 횟수']= len(give)

  return(result)




In [ ]:
simulation(n_join=100, join=50 ,out=10, claim = 0.057143 , T=10 , a0=1000 , unit=100 , c=20  , up=1.5, down=-3.48)

{'보험사 자본금': 1084.2820152230129, '평균지급액': 352.7389071236302, '지급 횟수': 3}

In [53]:
def model(num,param):
  result = pd.DataFrame()
  asset= []
  give = []
  for i in range(num):
    simul = simulation(n_join=100, join=50 ,out=10, claim = param['lambda'] , T=10 , a0=1000 , unit=100 , c=20  , up=param['updown'][1], down=param['updown'][0])
    asset.append(simul['보험사 자본금'])
    give.append(simul['평균지급액'])
  result['기준지수']=[param['updown'][1]]
  result['한도지수']=[param['updown'][0]]
  result['평균 보험사 자본금'] = [np.mean(asset)]
  result['평균 지급액']= [np.mean(give)]

  return result

def opt(where,grid,num,which):
  param = region(x=where,y=grid[0],fruit=which)
  result= model(num,param = param)
  result['low']= [grid[0][0]]
  result['high']= [grid[0][1]]
  result['지역']=[where]

  for i in range(1,len(grid)):
    param = region(x=where,y=grid[i],fruit=which)
    row =model(num,param = param)
    row['low']= [grid[i][0]]
    row['high']= [grid[i][1]]
    row['지역']=[where]
    result = pd.concat(([result, row]) ,axis =0)

  result = result.reset_index()
  return(result)

In [27]:
print(region(x='진주',y = [0.1,0.35],fruit = 'pear365'))
print(region(x='천안',y = [0.1,0.35],fruit = 'pear365'))
print(region(x='전주',y = [0.1,0.35],fruit = 'peach365'))
print(region(x='춘천',y = [0.1,0.35],fruit = 'peach365'))
print(region(x='대구',y = [0.1,0.35],fruit = 'peach365'))

{'lambda': 0.11538461538461539, 'updown': [-6.0, -1.0]}
{'lambda': 0.0, 'updown': [-7.4, -2.8]}
{'lambda': 0.14285714285714285, 'updown': [-4.1, 0.6599999999999994]}
{'lambda': 0.0, 'updown': [-8.04, -2.9]}
{'lambda': 0.05714285714285714, 'updown': [-3.4799999999999995, 1.5]}


In [54]:
opt(where='진주',grid=grid,num = 1000,which ='pear365')

,index,기준지수,한도지수,평균 보험사 자본금,평균 지급액,low,high,지역
0,0,-4.755,-6.0,2188.079636,0.000000,0.1,0.15,진주
1,0,-3.480,-6.0,2188.123266,0.000000,0.1,0.20,진주
2,0,-2.350,-6.0,1760.438647,167.908075,0.1,0.25,진주
3,0,-1.710,-6.0,1203.766455,211.254153,0.1,0.30,진주
4,0,-1.000,-6.0,451.310898,248.250268,0.1,0.35,진주
5,0,-0.200,-6.0,141.714173,293.212134,0.1,0.40,진주
6,0,0.335,-6.0,-2174.605680,314.704485,0.1,0.45,진주
7,0,0.900,-6.0,-3275.964594,344.192390,0.1,0.50,진주
8,0,1.700,-6.0,-5648.811886,383.660971,0.1,0.55,진주
9,0,2.600,-6.0,-7809.449571,429.796711,0.1,0.60,진주


In [55]:
opt(where='천안',grid=grid,num = 1000,which ='pear365')

,index,기준지수,한도지수,평균 보험사 자본금,평균 지급액,low,high,지역
0,0,-5.95,-7.4,2194.122048,0.000000,0.1,0.15,천안
1,0,-5.10,-7.4,2189.551749,0.000000,0.1,0.20,천안
2,0,-4.15,-7.4,2187.104807,0.000000,0.1,0.25,천안
3,0,-3.70,-7.4,2190.431284,0.000000,0.1,0.30,천안
4,0,-2.80,-7.4,2187.453049,0.000000,0.1,0.35,천안
5,0,-2.00,-7.4,2188.011686,0.000000,0.1,0.40,천안
6,0,-1.15,-7.4,2193.119541,0.000000,0.1,0.45,천안
7,0,-0.40,-7.4,1371.686427,319.531460,0.1,0.50,천안
8,0,0.50,-7.4,-564.256111,394.079814,0.1,0.55,천안
9,0,1.50,-7.4,-2960.648574,449.132143,0.1,0.60,천안


In [58]:
opt(where='전주',grid=grid,num = 1000,which ='peach365')

,index,기준지수,한도지수,평균 보험사 자본금,평균 지급액,low,high,지역
0,0,-2.90,-4.1,2186.049105,0.000000,0.1,0.15,전주
1,0,-1.96,-4.1,2190.801747,0.000000,0.1,0.20,전주
2,0,-1.00,-4.1,2187.436774,0.000000,0.1,0.25,전주
3,0,-0.10,-4.1,2195.276693,0.000000,0.1,0.30,전주
4,0,0.66,-4.1,164.263642,236.168227,0.1,0.35,전주
5,0,1.54,-4.1,-1181.711046,281.672466,0.1,0.40,전주
6,0,2.20,-4.1,-3698.404263,316.953621,0.1,0.45,전주
7,0,2.80,-4.1,-5993.521102,343.194937,0.1,0.50,전주
8,0,3.30,-4.1,-10382.632314,366.880551,0.1,0.55,전주
9,0,4.06,-4.1,-16374.132145,407.091949,0.1,0.60,전주


In [59]:
opt(where='춘천',grid=grid,num = 1000,which ='peach365')

,index,기준지수,한도지수,평균 보험사 자본금,평균 지급액,low,high,지역
0,0,-6.700,-8.04,2189.701921,0.000000,0.1,0.15,춘천
1,0,-5.640,-8.04,2186.644779,0.000000,0.1,0.20,춘천
2,0,-4.500,-8.04,2187.064226,0.000000,0.1,0.25,춘천
3,0,-3.600,-8.04,2189.937741,0.000000,0.1,0.30,춘천
4,0,-2.900,-8.04,2186.981523,0.000000,0.1,0.35,춘천
5,0,-2.300,-8.04,2193.549565,0.000000,0.1,0.40,춘천
6,0,-1.600,-8.04,2192.606599,0.000000,0.1,0.45,춘천
7,0,-1.150,-8.04,2187.666509,0.000000,0.1,0.50,춘천
8,0,-0.500,-8.04,2190.740850,0.000000,0.1,0.55,춘천
9,0,0.580,-8.04,1261.133875,380.857242,0.1,0.60,춘천


In [60]:
opt(where='대구',grid=grid,num = 1000,which ='peach365')

,index,기준지수,한도지수,평균 보험사 자본금,평균 지급액,low,high,지역
0,0,-2.10,-3.48,2189.798692,0.000000,0.1,0.15,대구
1,0,-1.00,-3.48,2188.933891,0.000000,0.1,0.20,대구
2,0,0.05,-3.48,2191.647157,0.000000,0.1,0.25,대구
3,0,0.90,-3.48,2190.173347,0.000000,0.1,0.30,대구
4,0,1.50,-3.48,1320.812452,242.128780,0.1,0.35,대구
5,0,2.28,-3.48,-709.363232,290.108007,0.1,0.40,대구
6,0,3.19,-3.48,-2364.469870,335.378588,0.1,0.45,대구
7,0,3.90,-3.48,-5997.355914,368.708333,0.1,0.50,대구
8,0,4.71,-3.48,-14000.870305,410.563314,0.1,0.55,대구
9,0,5.42,-3.48,-23429.443641,443.773130,0.1,0.60,대구
